<b>Step 1</b> - Importing libraries

In [9]:
import pandas as pd

<b>Step 2</b> - importing data

In [11]:
Data = pd.read_csv("WIDS Dataset_2020_Adj.csv")

In [13]:
Data[:5] #checking if the dataset was imported successfully

,X,Date,Zone1Position,Zone2Position,Zone3Position,SKU,Zone1_Row_Num,Zone1_Col_Num,Zone2_Row_Num,Zone2_Col_num,...,Zone2_Humidity_Max,Zone2_Humidity_Range,Zone3_Humidity_Avg,Zone3_Humidity_Min,Zone3_Humidity_Max,Zone3_Humidity_Range,Block_Num,Block_Position,Block_Orientation,Result_Type
0,1,09/01/2020,3.0,2.0,3.0,A001,1.0,3.0,1.0,2.0,...,61.80,10.0,39.19,33.19,45.19,12.0,1000.0,1.0,1,Defect_1
1,2,03/12/2019,8.0,4.0,3.0,A001,2.0,4.0,2.0,2.0,...,58.12,16.0,48.50,40.50,56.50,16.0,1003.0,1.0,1,Defect_1
2,3,21/11/2019,4.0,2.0,3.0,B003,1.0,4.0,1.0,2.0,...,50.97,10.0,52.20,46.20,58.20,12.0,1001.0,4.0,1,PASS
3,4,22/11/2019,6.0,3.0,1.0,B003,2.0,2.0,2.0,1.0,...,59.13,4.0,39.29,31.29,47.29,16.0,1003.0,2.0,1,PASS
4,5,23/12/2019,3.0,2.0,3.0,B003,1.0,3.0,1.0,2.0,...,65.80,18.0,45.37,42.37,48.37,6.0,1002.0,1.0,1,PASS


<b>Observation 1</b>: What is X? Is X the Index of the dataset?

<b>Step 3</b> - Exploring the data columns' name and the type of data in each column

In [23]:
Data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75750 entries, 0 to 75749
Data columns (total 53 columns):
X                         75750 non-null int64
Date                      75707 non-null object
Zone1Position             75712 non-null float64
Zone2Position             75706 non-null float64
Zone3Position             75713 non-null float64
SKU                       75711 non-null object
Zone1_Row_Num             75715 non-null float64
Zone1_Col_Num             75708 non-null float64
Zone2_Row_Num             75704 non-null float64
Zone2_Col_num             75716 non-null float64
Zone3_Row_Num             75713 non-null float64
Zone3_Col_Num             75714 non-null float64
Zone1_Left_Block_Bin      75708 non-null float64
Zone1_Right_Block_Bin     75710 non-null float64
Zone1_Area                75709 non-null object
Zone3_Area                75718 non-null object
Zone1_Dur                 75713 non-null float64
Zone2_Dur                 75708 non-null float64
Zone3_Dur      

<b>Observation 2:</b> Comparing this information to the <i>Data dictionary</i>, the following variables need to be reformated:
- <i>Zone1Position</i>, <i>Zone2Position</i> and <i>Zone3Position</i> - from float to categorical variables

<b>Step 4</b> - Identifying the number and percentage of missing data per each column

In [26]:
print("Number of missing values per each column:")
print(Data.isnull().sum())

Number of missing values per each column:
X                          0
Date                      43
Zone1Position             38
Zone2Position             44
Zone3Position             37
SKU                       39
Zone1_Row_Num             35
Zone1_Col_Num             42
Zone2_Row_Num             46
Zone2_Col_num             34
Zone3_Row_Num             37
Zone3_Col_Num             36
Zone1_Left_Block_Bin      42
Zone1_Right_Block_Bin     40
Zone1_Area                41
Zone3_Area                32
Zone1_Dur                 37
Zone2_Dur                 42
Zone3_Dur                 47
Zone1_Out_Zone2_In_Dur    43
Zone1_Out_Zone3_In_Dur    42
Zone2_Out_Zone3_In_Dur    41
Zone1_In_Zone3_Out_Dur    34
Zone1_In_Zone2_Out_Dur    44
Zone2_In_Zone3_Out_Dur    39
Zone1_Temp_Avg            52
Zone1_Temp_Min            46
Zone1_Temp_Max            39
Zone1_Temp_Range          41
Zone2_Temp_Avg            51
Zone2_Temp_Min            53
Zone2_Temp_Max            37
Zone2_Temp_Range          37
Z

In [28]:
print("Percentage of missing data in each column:")
sum_miss_data = (Data.isnull().sum()) / len(Data)
print(sum_miss_data)

Percentage of missing data in each column:
X                         0.000000
Date                      0.000568
Zone1Position             0.000502
Zone2Position             0.000581
Zone3Position             0.000488
SKU                       0.000515
Zone1_Row_Num             0.000462
Zone1_Col_Num             0.000554
Zone2_Row_Num             0.000607
Zone2_Col_num             0.000449
Zone3_Row_Num             0.000488
Zone3_Col_Num             0.000475
Zone1_Left_Block_Bin      0.000554
Zone1_Right_Block_Bin     0.000528
Zone1_Area                0.000541
Zone3_Area                0.000422
Zone1_Dur                 0.000488
Zone2_Dur                 0.000554
Zone3_Dur                 0.000620
Zone1_Out_Zone2_In_Dur    0.000568
Zone1_Out_Zone3_In_Dur    0.000554
Zone2_Out_Zone3_In_Dur    0.000541
Zone1_In_Zone3_Out_Dur    0.000449
Zone1_In_Zone2_Out_Dur    0.000581
Zone2_In_Zone3_Out_Dur    0.000515
Zone1_Temp_Avg            0.000686
Zone1_Temp_Min            0.000607
Zone1_Temp_M

<b>Observation 3</b>: The percentage of missing data is under 1% -> very low

<b>Step 5</b> - Data summary

In [16]:
Data.describe() #

,X,Zone1Position,Zone2Position,Zone3Position,Zone1_Row_Num,Zone1_Col_Num,Zone2_Row_Num,Zone2_Col_num,Zone3_Row_Num,Zone3_Col_Num,...,Zone2_Humidity_Min,Zone2_Humidity_Max,Zone2_Humidity_Range,Zone3_Humidity_Avg,Zone3_Humidity_Min,Zone3_Humidity_Max,Zone3_Humidity_Range,Block_Num,Block_Position,Block_Orientation
count,75750.000000,75712.000000,75706.000000,75713.000000,75715.000000,75708.000000,75704.000000,75716.000000,75713.000000,75714.000000,...,75710.000000,75703.000000,75699.000000,75710.000000,75711.000000,75715.000000,75715.000000,75708.000000,75711.000000,75750.0
mean,37505.877043,4.499511,2.499194,3.494605,1.498976,2.503223,1.498930,1.501189,1.497590,2.001704,...,45.713429,56.559406,10.847541,44.602891,40.068735,49.137194,9.068533,1003.509788,2.486746,1.0
std,21651.573231,2.291902,1.120779,1.705865,0.500002,1.113681,0.500002,0.500002,0.499997,0.817494,...,5.377477,5.381292,6.409238,5.504732,5.758575,5.776715,3.445690,2.292423,1.116384,0.0
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,30.260000,43.260000,2.000000,33.710000,25.710000,35.710000,4.000000,1000.000000,1.000000,1.0
25%,18760.250000,2.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,...,41.970000,53.010000,8.000000,41.570000,36.400000,45.400000,6.000000,1002.000000,1.000000,1.0
50%,37505.500000,4.000000,2.000000,3.000000,1.000000,3.000000,1.000000,2.000000,1.000000,2.000000,...,45.740000,56.870000,10.000000,44.360000,39.910000,48.940000,8.000000,1004.000000,2.000000,1.0
75%,56259.750000,7.000000,4.000000,5.000000,2.000000,3.000000,2.000000,2.000000,2.000000,3.000000,...,49.460000,60.300000,16.000000,48.320000,43.520000,52.710000,12.000000,1006.000000,3.000000,1.0
max,75000.000000,8.000000,4.000000,6.000000,2.000000,4.000000,2.000000,2.000000,2.000000,3.000000,...,60.130000,73.130000,24.000000,60.240000,58.240000,68.240000,16.000000,1007.000000,4.000000,1.0
